In [4]:
import pandas as pd
import math
from decimal import *
import numpy as np

In [5]:
dfAden = pd.read_csv('../data/aden.csv',sep=';')

In [6]:
dfAden.columns

Index([u'Unnamed: 0', u'Annee (0)', u'jour (0)', u'mois (0)',
       u'dist cote (nq)', u'zone', u'lat (0)', u'long(0)', u'type(0)',
       u'pavillon', u'statut de l'attaque(0)', u'type d'attaque(0)',
       u'Nombre P  Min', u'Nombre P  max', u'nombre embarcation',
       u'Vol a bord', u' Kidnapping', u'blessees', u' meurtre', u'Knives ',
       u'RPG', u'descriptif', u'latitude', u'longitude', u'quinzaine_week',
       u'newDescriptif', u'Guns', u'mousson', u'distance au port',
       u'distance_shorline', u'lat_area', u'lon_area'],
      dtype='object')

# Affectation des zones

In [141]:
min_lat=-10
max_lat=31
min_lon=31
max_lon=78

#un degre de latitude "vaut"  111,11 km
accuracy_km=100
accuracy_lat=accuracy_km/111.11

diff_lat=max_lat-min_lat
diff_lon=max_lon-min_lon

path_lat=float(diff_lat)/float(accuracy_lat)

latitude_grid=np.linspace(min_lat, max_lat, path_lat)
longitude_grid=[]
longitude_grid.append(min_lon)

tab=[]
for i in range(0,len(latitude_grid)-1):
    mean_lat=float(latitude_grid[i]+latitude_grid[i+1])/2
    
    #print mean_lat
    #print math.cos(math.radians(mean_lat))
    km_lon=111.11*math.cos(math.radians(mean_lat))
    accuracy_lon=accuracy_km/km_lon
    
    tab.append(accuracy_lon)
    
    currentLon=longitude_grid[i]+accuracy_lon
    longitude_grid.append(currentLon)
    #print 111.11*math.cos(math.radians(mean_lat))

    
#on complete jusqu'a 78
maxCurrent=max(longitude_grid)
meanPath=np.mean(tab)
while maxCurrent<78:
    maxCurrent=max(longitude_grid)
    longitude_grid.append(maxCurrent+meanPath)
    
    
print len(latitude_grid)
print len(longitude_grid)
print latitude_grid
print longitude_grid


45
53
[-10.          -9.06818182  -8.13636364  -7.20454545  -6.27272727
  -5.34090909  -4.40909091  -3.47727273  -2.54545455  -1.61363636
  -0.68181818   0.25         1.18181818   2.11363636   3.04545455
   3.97727273   4.90909091   5.84090909   6.77272727   7.70454545
   8.63636364   9.56818182  10.5         11.43181818  12.36363636
  13.29545455  14.22727273  15.15909091  16.09090909  17.02272727
  17.95454545  18.88636364  19.81818182  20.75        21.68181818
  22.61363636  23.54545455  24.47727273  25.40909091  26.34090909
  27.27272727  28.20454545  29.13636364  30.06818182  31.        ]
[31, 31.912614743237555, 32.822863634917184, 33.730998480299164, 34.637268219262715, 35.541919276934024, 36.44519590596541, 37.34734052164988, 38.248594031010946, 39.1491961569706, 40.049385758668464, 40.94940114898154, 41.849480410277394, 42.74986170942264, 43.65078361306438, 44.5524854042037, 45.45520740108823, 46.35919127946465, 47.26468039925254, 48.171920136727636, 49.08115822333648, 49.9926

In [142]:
def affectation_zone(latitude,longitude,latitude_grid,longitude_grid,lat_bool):
    #print 'latitude : '+ str(latitude)
    #print 'longitude : '+ str(longitude)
    
    for i in range(0,len(latitude_grid)-1):
        if latitude<latitude_grid[i+1] and latitude>latitude_grid[i]:
            idx_zone_lat=i
            
    for i in range(0,len(longitude_grid)-1):
        if longitude<longitude_grid[i+1] and longitude>longitude_grid[i]:
            idx_zone_lon=i

    #print idx_zone_lon
    #print idx_zone_lat
    if lat_bool:
        return idx_zone_lat
    else:
        return idx_zone_lon

In [143]:
latitude=9.91666666667
longitude=76.0833333333
idx_zone_lat=affectation_zone(latitude,longitude,latitude_grid,longitude_grid,True)
idx_zone_lon=affectation_zone(latitude,longitude,latitude_grid,longitude_grid,False)
print idx_zone_lat
print idx_zone_lon

21
48


In [144]:
dfAden['lat_area'] = dfAden.apply(lambda row: affectation_zone(row['latitude'],row['longitude'],latitude_grid,longitude_grid,True), axis=1)
dfAden['lon_area'] = dfAden.apply(lambda row: affectation_zone(row['latitude'],row['longitude'],latitude_grid,longitude_grid,False), axis=1)

In [145]:
dfAden['Annee (0)'] = dfAden['Annee (0)'].astype(int)
dfAden['jour (0)'] = dfAden['jour (0)'].astype(int)
dfAden['mois (0)'] = dfAden['mois (0)'].astype(int)

In [146]:
#dfAden.columns[0]
columnTodelete=filter(lambda x:'Unna' in x, dfAden.columns)

dfAden['descriptif']=dfAden.apply(lambda row: row['descriptif'].replace('\r', ''), axis=1)
dfAden['descriptif']=dfAden.apply(lambda row: row['descriptif'].replace('\r', ''), axis=1)
#print columnTodelete
#for col in columnTodelete:
#    print col
#    dfAden = dfAden.drop('Unnamed: 0')

In [147]:
dfAden.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Annee (0)', u'jour (0)', u'mois (0)',
       u'dist cote (nq)', u'zone', u'lat (0)', u'long(0)', u'type(0)',
       u'pavillon', u'statut de l'attaque(0)', u'type d'attaque(0)',
       u'Nombre P  Min', u'Nombre P  max', u'nombre embarcation',
       u'Vol a bord', u' Kidnapping', u'blessees', u' meurtre', u'Knives ',
       u'RPG', u'descriptif', u'latitude', u'longitude', u'quinzaine_week',
       u'newDescriptif', u'Guns', u'mousson', u'distance au port',
       u'distance_shorline', u'lat_area', u'lon_area'],
      dtype='object')

In [148]:
dfAden.to_csv("../data/aden.csv",sep=';')